In [37]:
import agentpy as ap
import matplotlib.pyplot as plt
import IPython
import random
import itertools
import json
from http.server import HTTPServer, BaseHTTPRequestHandler

In [38]:
# Type of move 
LEFT_RIGHT_MOVE = 1
RIGHT_LEFT_MOVE = 2
UP_DOWN_MOVE = 3
DOWN_UP_MOVE = 4


VALID_MOVES = {
    LEFT_RIGHT_MOVE: (0, 1),
    RIGHT_LEFT_MOVE: (0, -1),
    UP_DOWN_MOVE: (1,0),
    DOWN_UP_MOVE: (-1,0)
}

# Colors
CAR_COLOR = 1
YELLOW_COLOR = 101
GREEN_COLOR = 102
RED_COLOR = 103

ID = 0


json_string =""

In [39]:
class CarAgent(ap.Agent):
    def setup(self):
        # return value of the type of move
        self.current_move_type = random.choice(list(VALID_MOVES.keys()))
        self.id = 0
        self.x=0
        self.y=0
        self.z=0
    
    def get_move(self):
        # Allows cars to randomize speed
        if random.uniform(0,1)>0.2:
            # Returns the coordinate of the choosen move
            return VALID_MOVES[self.current_move_type]
        return(0,0)


In [40]:
class LightAgent(ap.Agent):
    def setup(self):

        # Iitilize the default color
        self.current_color = YELLOW_COLOR
        
        # Extra colors
        self.green_time = 0
        self.red_time = 0

    def start_light_plan(self, time_car):
        #If it's different from green and red time > 0
        if not self.is_green() and self.red_time > 0:
            self.green_time = time_car
            self.red_time = 0
            self.current_color = GREEN_COLOR
    def change_to_red(self, time_red):
        if self.current_color != RED_COLOR:
            self.current_color = RED_COLOR
            self.red_time = time_red
    def is_green(self):
        return self.current_color == GREEN_COLOR
    def change_to_yellow(self):
        self.green_time = 0
        self.red_time = 0

        self.current_color = YELLOW_COLOR

In [41]:
location_lights = itertools.product(range(5,50,5), range(10,100,10))
len(list(location_lights))

81

In [42]:
class MyHandler(BaseHTTPRequestHandler):
    def do_GET(self,json_string):
        response=json_string
        self.send_response(200)
        # send response headers
        self.end_headers()
        # send the body of the response
        self.wfile.write(bytes(response, "utf-8"))

httpd = HTTPServer(('localhost', 8000), MyHandler)
httpd.serve_forever()

In [43]:
class StreetModel(ap.Model):
    def setup(self):
        self.x_car =[]
        self.y_car =[]
        self.matrix =[]
        #Create light agents
        
        #Create first the X lights
        x_separation = self.p.lights_separation["X"]
        y_separation = self.p.lights_separation["Y"]

        x_valid = range(x_separation, self.p.size["X"], x_separation)
        y_valid = range(y_separation, self.p.size["Y"], y_separation)

        lights_location = list(itertools.product(y_valid, x_valid))
        n_lights = len(lights_location)

        self.lights = ap.AgentList(self, n_lights, LightAgent)
        self.lights.is_car = False
        self.lights.type_agent = YELLOW_COLOR

        self.cars = ap.AgentList(self, int(self.p["cars"]), CarAgent)
        iterator = 0

        self.current_car_positions = []

        for car in self.cars:
            car.id = iterator
            iterator += 1
            carObject = {
                    'id': car.id,
                    'x':0,
                    'y':0,
                }
            self.current_car_positions.append(carObject)
        self.cars.is_car = True
        self.cars.type_agent = CAR_COLOR

        #create grid (street)
        self.street = ap.Grid(self, [self.p["size"]["Y"], self.p["size"]["X"]], track_empty=True, torus=True)

        #add the lights agents
        self.street.add_agents(self.lights, lights_location)
        
        #randomly add cars in the "valid street" (where there are lights)
        valid_car_location = [(0, x) for x in x_valid] +  [(y, 0) for y in y_valid]
        initial_car_location = [random.choice(valid_car_location) for _ in self.cars]
        

        for car, initial_location in zip(self.cars, initial_car_location):
            if initial_location[0] == 0:
                #up/down
                car.current_move_type = random.choice([UP_DOWN_MOVE, DOWN_UP_MOVE])
            else:
                #left/right
                car.current_move_type = random.choice([LEFT_RIGHT_MOVE, RIGHT_LEFT_MOVE])
        
        self.street.add_agents(self.cars, initial_car_location)
    def step(self):
        iterator = 0
        for car in self.cars:
            coordinates_move = car.get_move()
            self.street.move_by(car, coordinates_move)

            # if(car.current_move_type==(0, 1)):
            #     car.x = car.x+1
            # elif(car.current_move_type==(0, -1)):
            #     car.x = car.x-1
            # if(car.current_move_type==(1,0)):
            #     car.y = car.y+1
            # elif(car.current_move_type==(-1, 0)):
            #     car.y = car.y-1
            
            if(coordinates_move==(0, 1)):
                car.x = car.x+1
                self.x_car += [car.x]
            elif(coordinates_move==(0, -1)):
                car.x = car.x-1
                self.x_car += [car.x]
            if(coordinates_move==(1,0)):
                car.y = car.y+1
                self.y_car += [car.y]
            elif(coordinates_move==(-1, 0)):
                car.y = car.y-1
                self.y_car += [car.y]
            
            self.current_car_positions[iterator]['x'] = car.x
            self.current_car_positions[iterator]['y'] = car.y

            iterator +=1
        json_string = json.dumps(self.current_car_positions)
        


        #print(json_string)
        #print(self.current_car_positions)
    
    def end(self):
        pass


In [44]:
# Define parameters

parameters = {
    'cars':4,
    'lights_separation': {'X': 5, 'Y':20},
    'size': {'X': 50, 'Y': 50}, # Height and length of the grid
    'steps': 15,
}

In [45]:

def animation_plot(model, ax):
    attr_grid = model.street.attr_grid('type_agent')
    color_dict = {CAR_COLOR:'#0000FF', YELLOW_COLOR:'#FFFF00', GREEN_COLOR:'#00FF00', RED_COLOR:'#FF0000' ,None:'#571150'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"Simulation of a forest fire\n"
                 f"Time-step: {model.t}, Trees left: ")
                 #f"{len(model.agents.select(model.agents.condition == 0))}")

fig, ax = plt.subplots()
model = StreetModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=1))


TypeError: send_response() missing 1 required positional argument: 'code'